<a href="https://www.kaggle.com/code/ashish7406/movie-recommendation-dataset?scriptVersionId=144709081" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movies_df=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits_df=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movies_df

In [ ]:
credits_df

In [ ]:
credits_df.head(10)

In [ ]:
movies_df.head()

In [ ]:
credits_df.head(1)['crew'].values

In [ ]:
credits_df.head(1)['cast'].values

#### Cast and Moveies table needed to merge with merge (using first dataframe of movies with credits dataframe(based on title) using df.merge(df1,on='column1')

In [ ]:
movies_df=movies_df.merge(credits_df,on='title')

Let us check the shape of our new dataset as dataframe 

In [ ]:
print(movies_df.shape)
print(credits_df.shape)

In [ ]:
print(movies_df.shape)

In [ ]:
movies_df.head(1)

### We need to keep such columns that are not required for analysis ,Content based recommendation builds. We will keep columns list that 
are required, we have to ensure that we create tags, if columns create tags. If budget you feel that it is not useful,whether you will like it or not as
recommendation, since it is not criteria to choose anything. Next is genrnes, as it is crucial
Next is home page that we will drop it, next is ID, which is useful, Movies poster it will be useful so it is crucial
Next is key words that is tags that are crucial
Original language, movies['original_language].value_counts(). since it is highyl imbalanced so we will avoid it. 
Original Title,could be hindi, so we will keep it
Overview , content similarity we can use for recommendation, summaries similar, so it is crucial.
Popularity could be , we need tags, it might not be useful
Production companies could be useful, but will not recommend based on production companies, 
Next is release date, mom dad might like it but again this is numerical column.
Revenue, it can be indicator it is numerical so it will not work
Movie time doesn't matter
status of movie, doenst matter
tag line of movie is textual but it could be weird so we will not include as overview will solve the purpose.
Title we kept
Movie ID is not required, Cast, Crew are crucial as it will play major role in recommending moveis.
Columns 



In [ ]:
movies_df['original_language'].value_counts()

In [ ]:
movies_df.head(5)

In [ ]:
# genres,keywords, title,id, overview, cast, crew

In [ ]:
movies_df.info()

In [ ]:
#How to use releasedate even though its numeric coumn
movies_df

In [ ]:
movies_df=movies_df[['id','title','genres','overview','cast','crew','keywords']]

New dataframe we will have Movie ID,Movie title, Movie tags
Tags are formed using 'overview','cast','crew','keywords'
Overview , cast, crew have keywords, cast(top three actors),


In [ ]:
movies_df.head()

In [ ]:
movies_df.isnull().sum()

In [ ]:
#dropping rows which are null
movies_df.dropna(inplace=True)

In [ ]:
movies_df.head(1)['crew'].values


In [ ]:
movies_df.isnull().sum()

## Find out duplicated rows

In [ ]:
#Validate if there are any duplcicated values
movies_df.duplicated().sum()

In [ ]:
movies_df.head(5)['genres'].values

In [ ]:
movies_df.iloc[0].genres
#This is in dictionary format and we have to put this in the format as['Action, 'Adventure', 'Fantasy'] and we need to create  helper function

In [ ]:
import ast
def convert(obj):
    L=[]
    obj=ast.literal_eval(obj)
    for i in obj:
        L.append(i['name'])
    return L
    

In [ ]:
movies_df['genres'].apply(convert)

In [ ]:
#Creating list of items, and iterate it
import ast
for i in ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'):
    print(i['name'])

In [ ]:
ast.literal_eval('[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}]')

In [ ]:
#Genres column with movies name
movies_df['genres']=movies_df['genres'].apply(convert)

In [ ]:
import ast
def convertKeywords(obj):
    L=[]
    obj=ast.literal_eval(obj)
    c=0
    for i in obj:
        if c != 3:
            L.append(i['name'])
            c+=1
        else:
            break
    return L
    

In [ ]:
#We will reflect the same for keywords
movies_df['keywords']=movies_df['keywords'].apply(convert)

In [ ]:
movies_df.head(5)

In [ ]:
convertKeywords

In [ ]:
movies_df['cast'][0]

In [ ]:
movies_df['cast']=movies_df['cast'].apply(convertKeywords)

In [ ]:
movies_df

Problem with crew is that there are many dictionaries where job is director and we need name of that same dictionary

In [ ]:
import ast
def fetchDirectorsCrew(obj):
    L=[]
    obj=ast.literal_eval(obj)
    for i in obj:
        if i['job'] =='Director':
            L.append(i['name'])
            break            
    return L

In [ ]:
movies_df['crew']=movies_df['crew'].apply(fetchDirectorsCrew)

In [ ]:
movies_df.head(5)

In [ ]:
movies_df['overview'][0]

Converting overview into object

In [ ]:
movies_df['overview']=movies_df['overview'].apply(lambda x:x.split())

In [ ]:
movies_df.head(5)

We need to apply transformation and convert spaces by removing spaces Sam Worthington(2 entities and create 2 tags)
Sam mendis (it will create two different tags), Hence it will create same tags as Sam will remain same. But it might create confusion
We can create a aword and make as one entity, that will make it unique, It will choose it accordingly.

In [ ]:
movies_df['cast']=movies_df['cast'].apply(lambda castname : [name.replace(" ","")for name in castname] )
movies_df['keywords']=movies_df['keywords'].apply(lambda keywords : [name.replace(" ","")for name in keywords] )
movies_df['genres']=movies_df['genres'].apply(lambda genres : [name.replace(" ","")for name in genres] )
movies_df['crew']=movies_df['crew'].apply(lambda crew : [name.replace(" ","")for name in crew] )

In [ ]:
movies_df.head(10)

In [ ]:
movies_df['tags']=movies_df['overview']+movies_df['cast']+movies_df['crew']+movies_df['keywords']+movies_df['genres']

In [ ]:
movies_df

In [ ]:
movies_df_new=movies_df[['id','title','tags']]

In [ ]:
movies_df_new['tags']=movies_df_new['tags'].apply(lambda x: " ".join(x))

In [ ]:
movies_df_new

In [ ]:
movies_df_new['tags'][0]

In [ ]:
movies_df_new['tags']=movies_df_new['tags'].apply(lambda x: x.lower())

In [ ]:
movies_df_new.head()

**Apply text vectorization** : Text is converted into vectors , data is in the form of movieid , name and tags.

In [ ]:
movies_df_new['tags'][0]

In [ ]:
movies_df_new['tags'][1]